## 다양한 임베딩을 통한 모델 개발 및 결과
 - 비정형데이터 특성상 정확한 accruacy측정이 힘들어 영화목록중 3가지(봄날은 간다, 주토피아, 마션)의 추천결과를 확인함으로 각 임베딩 방식의 결과를 보았다.
 - 크게 5가지의 임벵딩(TFIDF, N-gram, DOC2vec LSA, LDA)방식으로 임벵딩 하여 결과를 보았다.


# TFIDF
- TF행렬에 IDF행렬을 곱해주어 가중치를 부여하는 TFIDF를 사용하여 높은 성과를 목표로 하기보다 어느정도 활용가치가 있는가 확인하기위해 사용함

In [2]:
df = pd.read_csv("./mecab_tokenized_content")

In [2]:
evaluate = pd.read_csv("./evaluate_label")

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Mecab

def tfidf_model(keyword):    
   
    vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')
    blog_contents = list(df.token_descirption)
    X = vectorizer.fit_transform(blog_contents)
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word
    sentence_ls = [sentence]
    test_vec = vectorizer.transform(sentence_ls)
    test_vec.toarray()
    recomen = cosine_similarity(test_vec, X)
    recomen_ind = recomen.argsort()[:, ::-1]
    recomendation = list(recomen_ind[:, :10][0])

    return recomendation

## TFIDF 추천 결과

In [25]:
tfidf_model_list = tfidf_model("자연 대나무 숲")

#### 영화: 봄날은 간다 결과

In [26]:

df.iloc[tfidf_model_list]

,NAME,description,token_descirption
284,대나무골 테마공원,국내여행담양 대나무골 테마공원 푸른 대나무의 향기메타세콰이아가로수길 소쇄원 ...,국내 여행담 양 대나무 골 테마 공원 푸른 대나무 의 향기 메타 세 콰 이아 가로...
531,아홉산숲,부산 살면서 가보고 싶던 아홉산숲을 마침내 다녀왔다 예약을 해야 하고 제한된 인원만...,부산 살 면서 가 보 고 싶 던 아홉 산 숲 을 마침내 다녀왔 다 예약 을 해야 ...
55,거제맹종죽테마공원,경상도거제맹종죽테마공원 거제도여행으로 첫코스 루프리텔캄 복사 이웃추가본문 ...,경상도 거제 맹종죽 테마 공원 거제 도 여행 으로 첫 코스 루프 리텔 캄 복사 이...
884,담양 소쇄원,전라남도 담양군월의 담양 여행 소쇄원과 광주호 호수생태원 복사 이웃추...,전라남도 담양군 월 의 담양 여행 소쇄원 과 광주호 호수 생태 원 복사 이웃 추가...
234,태화강 국가정원,국내여행울산 가볼만한곳 태화강 국가정원 양귀비 금영화 라벤더 따스한 봄 이매미 ...,국내 여행 울산 가 볼 만 한 곳 태화강 국가 정원 양귀비 금영화 라벤더 따스 한...
772,죽녹원,전라남도 담양군담양 죽녹원과 담양호 용마루길 복사 이웃추가본문 기타 기...,전라남도 담양군 담양 죽 녹원 과 담양호 용마루 길 복사 이웃 추가 본문 기타 기...
313,태화강,국내여행울산 가볼만한곳 태화강 국가정원 양귀비 금영화 라벤더 따스한 봄 이매미 ...,국내 여행 울산 가 볼 만 한 곳 태화강 국가 정원 양귀비 금영화 라벤더 따스 한...
684,담양 명옥헌 원림,전라도담양 명옥헌 원림 배롱나무의 아름다움 속으로 일상탈출 복사 이웃추가본...,전라도 담양 명 옥 헌 원림 배롱나무 의 아름다움 속 으로 일상 탈출 복사 이웃 ...
203,용인자연휴양림,경기도용인 자연휴양림 일부개방경기도 가볼만한곳 그린향기 복사 이웃추가본문 ...,경기도 용인 자연 휴양림 일부 개방 경기 도 가 볼 만 한 곳 그린 향기 복사 이...
584,국립 산음자연휴양림,박일가을 알록달록 단풍구경 양평 캠핑여행정선의 오토캠핑장 동강전망자연휴양...,박일 가을 알록달록 단풍 구경 양평 캠핑 여행 정선 의 오토캠핑 장 동강 전망 자...


In [27]:
recommand_evaluate = evaluate.iloc[tfidf_model_list]


In [10]:
from gensim.models import doc2vec
doc2vec_model = doc2vec.Doc2Vec.load("./wiki_pos_tokenizer_without_taginfo.doc2vec.model")

In [13]:
# 평가하기
def evaluate_(model, keyword):
    from gensim.models import doc2vec
    from scipy import spatial
    from konlpy.tag import Mecab

    # tokenize
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word

    # vectorize
    model.random.seed(0)
    fisrt_text = sentence
    vec1 = model.infer_vector(fisrt_text.split())
    similar = []
    for i, row in recommand_evaluate.iterrows():
        vec2 = model.infer_vector(row.label.split())
        similairty = spatial.distance.cosine(vec1, vec2)
        similar.append(similairty)
    results = [x for x in similar if x < 0.76 ]
    print(similar)
    return len(results)

#### 영화: 주토피아 결과

In [42]:
tfidf_model_list = tfidf_model("가족 아이 동물")
df.iloc[tfidf_model_list]


,NAME,description,token_descirption
145,두물머리 애벌레 생태학교,양평 두물머리 애벌레 생태학교날이 정말정말 좋은 오늘수요일 교시 하는 날인데ㅠㅠ현장...,양평 두 물머리 애벌레 생태 학교 날 이 정말 정말 좋 은 오늘 수요일 교시 하 ...
961,부천 동물테마파크 하이주,아이와 갈만한곳부천 아이와 갈만한 곳 부천 동물테마파크 하이주 러블리뭉치 ...,아이 와 갈만 한 곳 부천 아이 와 갈만 한 곳 부천 동물 테마파크 하이주 러블리...
207,돼지문화원,국내여행강원도여행 원주 돼지문화원 아기와 가볼만한 곳 별양 복사 이웃추...,국내 여행 강원 도 여행 원주 돼지 문화원 아기 와 가 볼 만 한 곳 별양 복사 ...
288,서울어린이대공원,년코로나 피해 나들이 서울어린이대공원 놀이공원가요 복사 이웃추가본문 기...,년 코 로 나 피해 나 들 이 서울 어린이 대공원 놀이공원 가요 복사 이웃 추가 ...
775,우농타조농장,가볼만한곳파주 가볼만한곳 우농타조농장 타조마을 어린이체험이 가득한 곳 복...,가 볼 만 한 곳 파주 가 볼 만 한 곳 우 농 타조 농장 타조 마을 어린이 체험...
287,서울대공원,육아일기서울대공원 아이와 함께 나들이 그레텔 복사 이웃추가본문 기타 기능...,육아 일기 서울대 공원 아이 와 함께 나 들 이 그레 텔 복사 이웃 추가 본문 기...
530,화성 베어팜,베어팜 시설 안내화성 베어팜 자연 체험 동물편 베어팜돌이 복사 이웃추가본...,베어 팜 시설 안내 화성 베어 팜 자연 체험 동물 편 베어 팜 돌 이 복사 이웃 ...
53,안성팜랜드,알뜰정보안성팜랜드 요즘 상황 답답해서 댕겨옴 쪼꼬 복사 이웃추가본문 기타...,알뜰 정보 안성 팜 랜드 요즘 상황 답답 해서 댕겨 옴 쪼 꼬 복사 이웃 추가 본...
347,퍼스트가든,나들이파주 자동차극장 가볼만한곳 퍼스트가든 반했다 김보송이 복사 이웃추가...,나 들 이 파주 자동차극장 가 볼 만 한 곳 퍼스트 가 든 반 했 다 김 보 송 ...
195,대구 달성공원,경북대구 여행코스대구수목원달성공원수성못 도심속 명소들 혼소풍 복사 이웃추가...,경북 대구 여행 코스 대구 수목원 달성 공원수 성 못 도심 속 명소 들 혼소 풍 ...


#### 영화 : 마션 결과

In [44]:
tfidf_model_list = tfidf_model("화성 우주 기지 생존 우주선 우주인")
df.iloc[tfidf_model_list]

,NAME,description,token_descirption
65,나로우주센터 우주과학관,년도 여행고흥여행 나로우주센터 우주과학관현답 복사 이웃추가본문 기타 기능번...,년 도 여행 고흥 여행 나 로 우주 센터 우주 과학관 현답 복사 이웃 추가 본문 ...
736,옥토끼우주센터,전국공지 강화도 옥토끼우주센터 솔직한 방문 후기 친절한 유박사 복사 이웃추...,전국 공지 강화 도 옥토끼 우주 센터 솔직 한 방문 후기 친절 한 유박 사 복사 ...
844,수원 화성 [유네스코 세계문화유산],수원화성유네스코 세계문화유산 수원화성 한 바퀴대교약졸 복사 이웃추가본문 기...,수원 화성 유네스코 세계 문화유산 수원 화성 한 바퀴 대교약졸 복사 이웃 추가 본...
51,별빛정원우주,아이와 함께 가기 좋은 곳서울근교 가볼만한 곳 이천 별빛정원 우주 항상웃는 ...,아이 와 함께 가 기 좋 은 곳 서울 근교 가 볼 만 한 곳 이 천 별빛 정원 우...
485,사천 항공우주박물관,생일이 같은 남매이야기사천 항공우주박물관 남해여행 아이랑 가볼만한곳 쌀몽맘 ...,생일 이 같 은 남매 이야기 사 천 항공 우주 박물관 남 해 여행 아이 랑 가 볼...
705,기아자동차 화성공장,토지화성 노진리 기아자동차 인근 공장부지 소개합니다 한올공인중개사사무소 복...,토지 화성 노진리 기아 자동차 인근 공장 부지 소개 합니다 한올 공인 중개사 사무...
106,화성 우리꽃식물원,경기도화성 가볼만한곳 화성 우리꽃식물원 미르아빠 복사 이웃추가본문 기타 기...,경기도 화성 가 볼 만 한 곳 화성 우리 꽃 식물원 미르 아빠 복사 이웃 추가 본...
8,화성 공룡알 화석지,국내여행 슝슝화성 공룡알 화석지 진짜 화석을 보고 왔어요 사나엘라 복사 이...,국내 여행 슝 슝 화성 공룡 알 화석 지 진짜 화석 을 보 고 왔 어요 사나 엘라...
530,화성 베어팜,베어팜 시설 안내화성 베어팜 자연 체험 동물편 베어팜돌이 복사 이웃추가본...,베어 팜 시설 안내 화성 베어 팜 자연 체험 동물 편 베어 팜 돌 이 복사 이웃 ...
558,남열해돋이해수욕장,국내여행고성 남열해돋이해수욕장낭만서프하우스 원룸형 펜션숙소박 후기 프레시 ...,국내 여행 고성 남 열 해돋이 해수욕장 낭 만 서프 하우스 원룸 형 펜션 숙소 박...


# N-gram
- 고전언어 모델중 하나인 N-gram방식을 사용하여 어느정도의 결과를 활용가치가 있는가를 확인하기 위해 사용해 보았다.

In [7]:
def ngram(text,i): 
    result = [] 
    for a in range(len(text) -i +1): 
        result.append(text[a:a+i]) 
    return result 
def getAccuracy(aText, bText, num): 
    aResult = ngram(aText,num) 
    bResult = ngram(bText,num) 
    cnt = 0 
    r = [] 
    for i in aResult: 
        for j in bResult: 
            if i == j: 
                cnt += 1 
                r.append(i) 
    return cnt / len(aResult), r



In [8]:
def ngram_recommand_list(keyword):
    from konlpy.tag import Mecab

    # tokenize
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word

    # vectorize
  
    similar = []
    for i, row in df.iterrows():
               
        accuracy, result = getAccuracy(sentence,row.token_descirption,3)
        similar.append(accuracy)
    
    return list(np.array(similar).argsort()[::-1][:10])

## N-gram 추천 결과

#### 영화 : 주토피아 결과

In [9]:
recommand_list = ngram_recommand_list("가족 아이 동물")
df.iloc[recommand_list]


,NAME,description,token_descirption
775,우농타조농장,가볼만한곳파주 가볼만한곳 우농타조농장 타조마을 어린이체험이 가득한 곳 복...,가 볼 만 한 곳 파주 가 볼 만 한 곳 우 농 타조 농장 타조 마을 어린이 체험...
832,서울 애니메이션센터,유아교육서울애니메이션센터 에서 살아있는동화로 창의력 키워봐요 후니로니맘 ...,유아 교육 서울 애니메이션 센터 에서 살 아 있 는 동화 로 창의력 키워 봐요 후...
218,스파플러스,대한민국 방방곡곡아이와 함께 이천 미란다호텔 헬로카봇 뉴캐릭터룸 스파플러스 이용...,대한민국 방방곡곡 아이 와 함께 이 천 미란다 호텔 헬로 카봇 뉴 캐릭터 룸 스파...
347,퍼스트가든,나들이파주 자동차극장 가볼만한곳 퍼스트가든 반했다 김보송이 복사 이웃추가...,나 들 이 파주 자동차극장 가 볼 만 한 곳 퍼스트 가 든 반 했 다 김 보 송 ...
53,안성팜랜드,알뜰정보안성팜랜드 요즘 상황 답답해서 댕겨옴 쪼꼬 복사 이웃추가본문 기타...,알뜰 정보 안성 팜 랜드 요즘 상황 답답 해서 댕겨 옴 쪼 꼬 복사 이웃 추가 본...
589,제주 노루생태관찰원,제주방방곡곡제주 노루생태관찰원은 노루와 아이가 교감하는 제주도 아이들과 가볼만한곳 ...,제주 방방곡곡 제주 노루 생태 관찰 원 은 노루 와 아이 가 교감 하 는 제주도 ...
605,산들소리수목원,육아일기산들소리수목원 아이랑 가기 너무 좋아요 뽀블링 복사 이웃추가본문 기...,육아일 기산 들 소리 수목원 아이 랑 가 기 너무 좋 아요 뽀 블링 복사 이웃 추...
75,부산영화체험박물관,안녕하세요 로키입니다방학시즌이죠 아이들이기다리던 시간이 아니였을까싶은데요 추워지다보...,안녕 하 세요 로키 입니다 방학 시즌 이 죠 아이 들 이 기다리 던 시간 이 아니...
802,용산가족공원,오늘도평화로운용산가족공원도심한가운데있는공원중에가장푸르르고새들이노래즐겨부르는곳주말에는...,오늘 도 평화 로운 용산 가족 공원 도 심한 가운데 있 는 공원 중 에 가장 푸르...
52,덕평공룡수목원,쑥쑥크는 예봄이아기랑 갈만한곳 덕평공룡수목원 간만에 즐거운 나들이 씬 복...,쑥쑥 크 는 예 봄 이 아기 랑 갈만 한 곳 덕평 공룡 수목원 간 만 에 즐거운 ...


#### 영화 : 봄날은 간다 결과

In [10]:
recommand_list = ngram_recommand_list("자연 대나무 숲")
df.iloc[recommand_list]

,NAME,description,token_descirption
284,대나무골 테마공원,국내여행담양 대나무골 테마공원 푸른 대나무의 향기메타세콰이아가로수길 소쇄원 ...,국내 여행담 양 대나무 골 테마 공원 푸른 대나무 의 향기 메타 세 콰 이아 가로...
699,울진 금강소나무 숲길,울진 금강 소나무 숲길금강소나무숲길 개 구간 산양과 자연보호 위해 매일 여명만 예...,울진 금강 소나무 숲길 금강 소나무 숲길 개 구간 산양 과 자연 보호 위해 매일 ...
3,원대리 자작나무 숲 (속삭이는 자작나무 숲),강원도 인제여행 원대리 자작나무숲 속삭이는 자작나무 숲 유니나 복사 이웃...,강원도 인제 여행 원대리 자작 나무숲 속삭이 는 자작나무 숲 유니 나 복사 이웃 ...
633,국립 청태산자연휴양림,놀자국립 청태산자연휴양림 그리고 눈썰매 연결잭 복사 이웃추가본문 기타 기능...,놀 자 국립 청태산 자연 휴양림 그리고 눈썰매 연결 잭 복사 이웃 추가 본문 기타...
748,국립 가리왕산자연휴양림,국립휴양림 가리왕산자영휴양림 둘러 보니지난번 정선 여행중 가리왕산자연휴양림 입구에 ...,국립 휴양림 가리 왕산 자영 휴양림 둘러 보 니 지난번 정선 여행 중 가리왕산 자...
7,국립 검마산자연휴양림,즐거운 환경 캠핑국립검마산자연휴양림 속 친환경캠핑 국립자연휴양림관리소 복...,즐거운 환경 캠핑 국립 검마산 자연 휴양림 속 친환경 캠핑 국립 자연 휴양림 관리...
203,용인자연휴양림,경기도용인 자연휴양림 일부개방경기도 가볼만한곳 그린향기 복사 이웃추가본문 ...,경기도 용인 자연 휴양림 일부 개방 경기 도 가 볼 만 한 곳 그린 향기 복사 이...
150,설악산국립공원(외설악),여행국내트레킹 수려한 경관자원의 설악산국립공원 서울춘천고속도로 복사 이웃...,여행 국내 트레 킹 수려 한 경관 자원 의 설악산 국립 공원 서울 춘천 고속도로 ...
884,담양 소쇄원,전라남도 담양군월의 담양 여행 소쇄원과 광주호 호수생태원 복사 이웃추...,전라남도 담양군 월 의 담양 여행 소쇄원 과 광주호 호수 생태 원 복사 이웃 추가...
580,국립 방장산자연휴양림,여행국립방장산자연휴양림박일 야챙이 복사 이웃추가본문 기타 기능지도로 보기 ...,여행 국립 방장산 자연 휴양림 박일 야 챙 이 복사 이웃 추가 본문 기타 기능 지...


#### 영화 : 마션 결과

In [11]:
recommand_list = ngram_recommand_list("화성 우주 기지 생존 우주선 우주인")
df.iloc[recommand_list]

,NAME,description,token_descirption
736,옥토끼우주센터,전국공지 강화도 옥토끼우주센터 솔직한 방문 후기 친절한 유박사 복사 이웃추...,전국 공지 강화 도 옥토끼 우주 센터 솔직 한 방문 후기 친절 한 유박 사 복사 ...
65,나로우주센터 우주과학관,년도 여행고흥여행 나로우주센터 우주과학관현답 복사 이웃추가본문 기타 기능번...,년 도 여행 고흥 여행 나 로 우주 센터 우주 과학관 현답 복사 이웃 추가 본문 ...
51,별빛정원우주,아이와 함께 가기 좋은 곳서울근교 가볼만한 곳 이천 별빛정원 우주 항상웃는 ...,아이 와 함께 가 기 좋 은 곳 서울 근교 가 볼 만 한 곳 이 천 별빛 정원 우...
485,사천 항공우주박물관,생일이 같은 남매이야기사천 항공우주박물관 남해여행 아이랑 가볼만한곳 쌀몽맘 ...,생일 이 같 은 남매 이야기 사 천 항공 우주 박물관 남 해 여행 아이 랑 가 볼...
844,수원 화성 [유네스코 세계문화유산],수원화성유네스코 세계문화유산 수원화성 한 바퀴대교약졸 복사 이웃추가본문 기...,수원 화성 유네스코 세계 문화유산 수원 화성 한 바퀴 대교약졸 복사 이웃 추가 본...
468,화성행궁,블로그서포터즈화성행궁에 봄이 왔대요 수원시청 복사 이웃추가본문 기타 기...,블로그 서포터 즈 화성행궁 에 봄 이 왔 대요 수원 시청 복사 이웃 추가 본문 기...
558,남열해돋이해수욕장,국내여행고성 남열해돋이해수욕장낭만서프하우스 원룸형 펜션숙소박 후기 프레시 ...,국내 여행 고성 남 열 해돋이 해수욕장 낭 만 서프 하우스 원룸 형 펜션 숙소 박...
8,화성 공룡알 화석지,국내여행 슝슝화성 공룡알 화석지 진짜 화석을 보고 왔어요 사나엘라 복사 이...,국내 여행 슝 슝 화성 공룡 알 화석 지 진짜 화석 을 보 고 왔 어요 사나 엘라...
705,기아자동차 화성공장,토지화성 노진리 기아자동차 인근 공장부지 소개합니다 한올공인중개사사무소 복...,토지 화성 노진리 기아 자동차 인근 공장 부지 소개 합니다 한올 공인 중개사 사무...
281,율암온천숯가마 테마파크,나의 일상 이야기경기도 화성 율암 겨울 아이들과 갈만한 곳율암온천 숯가마 테마파크 ...,나 의 일상 이야기 경기도 화성 율암 겨울 아이 들 과 갈만 한 곳 율 암 온천 ...


# DOC2Vec

## 1st. DOC2vec-블로그 글뭉치(PV-DM)

In [ ]:
from gensim.models import doc2vec
import sys
import multiprocessing


cores = multiprocessing.cpu_count()
# 파리미터설정
vector_size = 300
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100

dm = 1  # {0:dbow, 1:dmpv}

worker_count = cores  # 병렬프로세스수 설정
ROOT_DIR = "C:\\code\\"
inputfile = ROOT_DIR + "blog_text.txt"
modelfile = ROOT_DIR + "blog_text.doc2vec.model"
word2vec_file = modelfile + ".word2vec_format"
sentences = doc2vec.TaggedLineDocument(inputfile)
# voca 만들기
model = doc2vec.Doc2Vec(min_count=word_min_count, vector_size=vector_size,

                        alpha=0.025, min_alpha=0.025, seed=1234, workers=worker_count)

model.build_vocab(sentences)
# 문서 학습
model.train(sentences, epochs=model.iter, total_examples=model.corpus_count)
# 저장
model.save(modelfile)
model.save_word2vec_format(word2vec_file, binary=False)

In [26]:
doc2vec_model = doc2vec.Doc2Vec.load("./blog_text.doc2vec.model")

#### 문제점 : 블로그 글 명사추출 필요성
- 학습결과 자연과 가까운 단어를 보았을때 조사와 어미등이 많이 포함된것을 확인 할수 있다

In [29]:

doc2vec_model.most_similar("자연")

[('을', 0.9981570839881897),
 ('은', 0.9981531500816345),
 ('주', 0.9981382489204407),
 ('스카이', 0.9981375336647034),
 ('도', 0.9981339573860168),
 ('고', 0.9981338977813721),
 ('이', 0.9981322288513184),
 ('의', 0.9981301426887512),
 ('로', 0.9981218576431274),
 ('가', 0.9981217384338379)]

In [ ]:
# doc2vec 모델로 코사인유사도를 구한 거리를 짧은순 인덱스(10개)로 추출해주는 함수
def doc2vec_recommand_list(model, keyword):
    from gensim.models import doc2vec
    from scipy import spatial
    from konlpy.tag import Mecab

    # tokenize
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word

    # vectorize
    fisrt_text = sentence
    vec1 = model.infer_vector(fisrt_text.split())
    similar = []
    for i, row in df.iterrows():
        vec2 = model.infer_vector(row.token_descirption.split())
        similairty = spatial.distance.cosine(vec1, vec2)
        similar.append(similairty)

    return list(np.array(similar).argsort()[:10])

#### 영화 봄날은 간다 결과
- 추천결과 또한 좋은 추천을 하지 않다고 판단됨.

In [28]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "자연 대나무 숲")
df.iloc[recommand_list]

,NAME,description,token_descirption
583,팔공산도립공원(갓바위지구),축제이야기팔공산도립공원 갓바위 산행 경상북도 보이소 복사 이웃추가본문 기...,축제 이야기 팔 공산 도 립 공원 갓바위 산행 경상북도 보 이소 복사 이웃 추가 ...
113,가우도 출렁다리,달달한 여행지가우도 출렁다리 바다를 보며 걸어봐 프림커피 복사 이웃추가본문...,달달 한 여행지 가 우도 출렁다리 바다 를 보 며 걸 어 봐 프림 커피 복사 이웃...
352,도마령,킹콩의 등산이야기민주지산 등산코스 도마령 각호산 민주지산 물한계곡 등산 후기 ...,킹콩 의 등산 이야기 민주 지산 등산 코스 도마 령 각호산 민주지산 물 한 계곡 ...
388,대이작도,산과 함께대이작도 섬 트레킹 부아산 송이산 계남마을 하트해변 풀등은 산삼 ...,산 과 함께 대이작도 섬 트레 킹 부아 산 송이 산 계남 마을 하트 해변 풀등 은...
408,송지호 해안 서낭바위 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
681,내변산,가을풍경내변산 가을 산행 가을의 변산반도 국립공원 풍경바라기 복사 이웃추...,가을 풍경 내변산 가을 산행 가을 의 변산반도 국립 공원 풍경 바라기 복사 이웃 ...
680,적석산,산행경상남도 고성과 창원 경계의 적석산 산행 하세틱 복사 이웃추가본문 기타...,산행 경상남도 고성 과 창원 경계 의 적석산 산행 하세 틱 복사 이웃 추가 본문 ...
251,향적봉(덕유산),등산 전라북도에 위치한 설산이 이쁘다는 덕유산 향적봉 무주리조트 눈부신청춘아 ...,등산 전라북도 에 위치 한 설산 이 이쁘 다는 덕유산 향 적 봉 무주 리조트 눈부...
669,천태산,대 명산충북 영동 천태산 산행 옥새봉 카이사르 복사 이웃추가본문 기타 ...,대 명산 충북 영동 천태산 산행 옥새 봉 카이사르 복사 이웃 추가 본문 기타 기능...
504,가우도,여행맛집부모님과 떠나는 전라도 별미 웰빙 박 일 여행후기두륜산케이블카두륜산힐링로드가...,여행 맛집 부모 님 과 떠나 는 전라도 별미 웰빙 박 일 여행 후기 두륜산 케이블...


## 2nd. DOC2vec-블로그 글뭉치 명사(PV-DM)
- 위의 문제점을 바탕으로 글뭉치에서 명사만을 추출하여 DOC2Vec에 학습 시켜보았다.

In [13]:
from gensim.models import doc2vec

In [ ]:
from gensim.models import doc2vec
import sys
import multiprocessing


cores = multiprocessing.cpu_count()
# 파리미터설정
vector_size = 300
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100

dm = 1  # {0:dbow, 1:dmpv}

worker_count = cores  # 병렬프로세스수 설정
ROOT_DIR = "C:\\code\\"
inputfile = ROOT_DIR + "blog_text_nouns.txt"
modelfile = ROOT_DIR + "blog_text_noun.doc2vec.model"
word2vec_file = modelfile + ".word2vec_format"
sentences = doc2vec.TaggedLineDocument(inputfile)
# voca 만들기
model = doc2vec.Doc2Vec(min_count=word_min_count, vector_size=vector_size,

                        alpha=0.025, min_alpha=0.025, seed=1234, workers=worker_count)

model.build_vocab(sentences)
# 문서 학습
model.train(sentences, epochs=model.iter, total_examples=model.corpus_count)
# 저장
model.save(modelfile)
model.save_word2vec_format(word2vec_file, binary=False)

In [30]:
doc2vec_model = doc2vec.Doc2Vec.load("./blog_text_noun.doc2vec.model")

####  문제점 : 다른 글뭉치로 학습의 필요성
- 명사만으로 추출된 글뭉치로 자연이라는 단어의 유사어를 보았을때 단어가 추출됨
- 단어가 추출되었으나 근접한 단어를 보았을때 자연으로 연상되는 단어가 많이 보이지 않음

In [31]:

doc2vec_model.most_similar("자연")

[('애리', 0.9999583959579468),
 ('휴', 0.9999382495880127),
 ('워크', 0.9999368190765381),
 ('짚', 0.9999347925186157),
 ('축제', 0.9999337196350098),
 ('칠갑산', 0.9999336004257202),
 ('출렁다리', 0.9999319911003113),
 ('병방', 0.9999315142631531),
 ('코스', 0.9999303817749023),
 ('스카이', 0.9999299049377441)]

In [20]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "자연 대나무 숲")
df.iloc[recommand_list]

,NAME,description,token_descirption
583,팔공산도립공원(갓바위지구),축제이야기팔공산도립공원 갓바위 산행 경상북도 보이소 복사 이웃추가본문 기...,축제 이야기 팔 공산 도 립 공원 갓바위 산행 경상북도 보 이소 복사 이웃 추가 ...
408,송지호 해안 서낭바위 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
431,소돌아들바위공원,캠핑장 여행지 리뷰주문진 갈만한곳 소돌아들바위공원 들러볼 만하네 택사마 복...,캠 핑장 여행지 리뷰 주문진 갈만 한 곳 소 돌 아들 바위 공원 들러볼 만 하 네...
846,단양 고수동굴,여행충북단양여행스카이워크도담삼봉고수동굴다누리아쿠아리움 이효순 복사 이웃추가...,여행 충북 단양 여행 스카이 워크 도 담 삼 봉고 수동 굴다 누리 아쿠아리움 이효...
760,실리도,전국섬 트레킹창원의 마지막 끝섬인 실리도 둘레길 탐방 하세틱 복사 이웃추가...,전국 섬 트레 킹 창원 의 마지막 끝 섬인 실리 도 둘레길 탐방 하세 틱 복사 이...
500,수락계곡,날짜 토 날씨 수락계곡쪽 상부 능선에서는 바람이 꽤 시원하게 불었고 전반적으로...,날짜 토 날씨 수락 계곡 쪽 상부 능선 에서 는 바람 이 꽤 시원 하 게 불 었 ...
151,용천동굴 [유네스코 세계자연유산],관광지 투어제주 용천동굴 유네스코 세계자연유산 용암동굴제주도렌트카협동조합 ...,관광지 투어 제주 용천 동굴 유네스코 세계 자연 유산 용암 동굴 제주도 렌트카 협...
42,진관사계곡,국립공원 힐링산행 북한산 국립공원 응봉능선진관사 계곡 원점회귀 코스 국립공원 반달...,국립 공원 힐링 산행 북한 산 국립 공원 응봉 능선 진관사 계곡 원점 회귀 코스 ...
547,공주 송산리 고분군과 무령왕릉 [유네스코 세계유산],명소시설유네스코 세계유산 송산리고분군무령왕릉 흥미진진 공주 복사 이웃추가본...,명소 시설 유네스코 세계유산 송산리 고분군 무령왕릉 흥미 진진 공주 복사 이웃 추...
302,만어사,가볼만한곳신비로운 전설이 숨어있는 사찰 만어사 밀양시 복사 이웃추가본문 기...,가 볼 만 한 곳 신비 로운 전설 이 숨 어 있 는 사찰 만어사 밀양시 복사 이웃...


In [19]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "가족 아이 동물")
df.iloc[recommand_list]

,NAME,description,token_descirption
174,천곡황금박쥐동굴,국내여행강원도 동해 여행 천곡황금박쥐동굴 알차네요 커피소녀윤짱 복사 이웃추...,국내 여행 강원도 동해 여행 천곡 황금박쥐 동굴 알차 네요 커피 소녀 윤 짱 복사...
46,소창체험관,어디든좋아강화 소창체험관 아이와함께 지몬스터 복사 이웃추가본문 기타 기능...,어디 든 좋 아 강화 소창 체험 관 아이 와 함께 지 몬스터 복사 이웃 추가 본문...
834,충청남도 안전체험관,충청남도 안전체험관 다녀왔어요리청 복사 이웃추가본문 기타 기능지도로 보...,충청남도 안전 체험 관 다녀왔 어요 리청 복사 이웃 추가 본문 기타 기능 지도 로...
846,단양 고수동굴,여행충북단양여행스카이워크도담삼봉고수동굴다누리아쿠아리움 이효순 복사 이웃추가...,여행 충북 단양 여행 스카이 워크 도 담 삼 봉고 수동 굴다 누리 아쿠아리움 이효...
953,조랑말체험공원,제주도울릉도제주 조랑말체험공원 노란 봄꽃물결 초록바다 최양훈 복사 이웃추가...,제주도 울릉도 제주 조랑말 체험 공원 노란 봄 꽃물결 초록 바다 최 양훈 복사 이...
323,온달관광지,여행은 좋아라단양여행 아이들이 정말 좋아했던 온달국민관광지 온달동굴 불꽃남자 ...,여행 은 좋 아라 단양 여행 아이 들 이 정말 좋 아 했 던 온달 국민 관광지 온...
173,자수정동굴나라,여기저기언양 자수정동굴나라 다녀왔어요 쭈띠 복사 이웃추가본문 기타 기능번역...,여기저기 언양 자수정 동굴 나라 다녀왔 어요 쭈띠 복사 이웃 추가 본문 기타 기능...
417,전주전동성당,제주말구월 전주여행 전주한옥마을 전동성당 전일갑오 조점례피순대 길거리야 ...,제주 말 구월 전주 여행 전주 한옥마을 전동성당 전일 갑오 조점 례 피순대 길 거...
288,서울어린이대공원,년코로나 피해 나들이 서울어린이대공원 놀이공원가요 복사 이웃추가본문 기...,년 코 로 나 피해 나 들 이 서울 어린이 대공원 놀이공원 가요 복사 이웃 추가 ...
15,순천만 국가정원,국내여행전라남도순천 순천만습지 순천만 국가정원 마덕리꼬마 복사 이웃추가...,국내 여행 전라남도 순천 순천 만 습지 순천 만 국가 정원 마 덕리 꼬마 복사 이...


In [21]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "화성 우주 기지 생존 우주선 우주인")
df.iloc[recommand_list]

,NAME,description,token_descirption
752,진해내수면 환경생태공원,경상도진해 내수면환경생태공원 아이와 가볼만한 곳 룰루랄 복사 이웃추가본문 ...,경상도 진해 내수면 환경 생태 공원 아이 와 가 볼 만 한 곳 룰 루 랄 복사 이...
335,신풍 신천 바다목장,대한민국제주 신풍신천바다목장 월 초에는 귤 말리는 풍경을 볼 수 없어요 제이 ...,대한민국 제주 신풍 신천 바다 목장 월 초 에 는 귤 말리 는 풍경 을 볼 수 없...
469,안면도 미로공원,국내여행태안 가볼만한곳 안면도 미로공원 바람아래관광농원살려줘뷰스타 블랑 복...,국내 여행 태안 가 볼 만 한 곳 안면 도 미 로 공원 바람 아래 관광농원 살려줘...
262,속초엑스포유람선,여행속초놀거리 속초엑스포유람선 서울춘천고속도로 복사 이웃추가본문 기타 기...,여행 속초 놀 거리 속초 엑스포 유람선 서울 춘천 고속도로 복사 이웃 추가 본문 ...
839,무주리조트 관광곤도라,여행 후기무주리조트 관광곤도라 이용 어른왕자 복사 이웃추가본문 기타 기능...,여행 후기 무주 리조트 관광 곤도 라 이용 어른 왕자 복사 이웃 추가 본문 기타 ...
393,울산바위촬영휴게소,여행속초여행울산바위촬영휴게소 울산바위보기 울산바위 인제내린천휴게소 리따 복...,여행 속초 여행 울산바위 촬영 휴게소 울산 바위 보 기 울산바위 인제 내린천 휴게...
856,남원 큰엉해안,힐링여행남원큰엉해안경승지 한반도 지도가 유명하죠 올레감사 복사 이웃추가본문...,힐링 여행 남원 큰 엉 해 안경 승지 한반도 지도 가 유명 하 죠 올레 감사 복사...
190,대관령 황태덕장마을,강원도 평창 대관령 맛집 용평 황태덕장 인생 황태구이 만남 최봄가을 복사 ...,강원도 평창 대관령 맛집 용평 황태 덕장 인생 황태 구이 만남 최 봄 가을 복사 ...
295,"서울 구 벨기에영사관(현, 서울시립미술관 남서울생활미술관)",뚜벅이로 돌아다니기미술관이 된 구 벨기에 영사관 서울시립미술관 남서울 분관 취미밋...,뚜벅 이 로 돌아다니 기 미술관 이 된 구 벨 기 에 영사관 서울 시립 미술관 남...
261,동래온천 노천족탕,아름다운 대한민국동래 가볼만한곳 이색체험 동래온천노천족탕다솜 복사 이웃추가...,아름다운 대한민국 동래 가 볼 만 한 곳 이색 체험 동래 온 천노 천 족탕 다솜 ...


## 3rd. DOC2vec-위키피디아 글뭉치(PV-DM)
- 위의 문제점을 바탕으로 위키피디아 글뭉치를 활용하여 학습을 시켜보았다
- '자연'이라는 단어의 유사어를 보았을때 좋은 않은 결과를 보였지만 다른 단어를 보았을때 대게 학습이 잘 되어진 모습을 보였다.
- 문제점: 봄날은 간다, 주토피아의 결과는 양호하나 마션의 결과가 좋지 않음

In [40]:
from gensim.models import doc2vec
doc2vec_model = doc2vec.Doc2Vec.load("./wiki_pos_tokenizer_without_taginfo.doc2vec.model")

In [20]:
def doc2vec_recommand_list(model, keyword):
    from gensim.models import doc2vec
    from scipy import spatial
    from konlpy.tag import Mecab

    # tokenize
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word

    # vectorize
    fisrt_text = sentence
    vec1 = model.infer_vector(fisrt_text.split())
    similar = []
    for i, row in df.iterrows():
        vec2 = model.infer_vector(row.token_descirption.split())
        similairty = spatial.distance.cosine(vec1, vec2)
        similar.append(similairty)

    return list(np.array(similar).argsort()[:10])

In [47]:
doc2vec_model.most_similar("자연")

[('급작', 0.5269038677215576),
 ('부자연', 0.5126141905784607),
 ('게걸', 0.48660343885421753),
 ('억척', 0.4819581210613251),
 ('천연덕', 0.47750020027160645),
 ('우스꽝', 0.47534075379371643),
 ('불미', 0.47219717502593994),
 ('믿음직', 0.4649367332458496),
 ('고풍', 0.4630765914916992),
 ('변덕', 0.45336684584617615)]

In [46]:
doc2vec_model.most_similar("바다")

[('대양', 0.5360127687454224),
 ('육지', 0.5315563082695007),
 ('해안가', 0.4941416382789612),
 ('홍해', 0.486382395029068),
 ('외해', 0.48536455631256104),
 ('빙산', 0.48493021726608276),
 ('바닷속', 0.4816575050354004),
 ('바닷가', 0.4793645739555359),
 ('산호초', 0.47544974088668823),
 ('파도', 0.47487521171569824)]

#### 영화 : 봄날은 간다 결과

In [39]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "자연 대나무 숲")
df.iloc[recommand_list]

,NAME,description,token_descirption
474,충주자연호텔,안녕하세요 오늘은 첫 자전거 종주 돌아오는 길에 묵었던 충주 자연 호텔 사우나에 대...,안녕 하 세요 오늘 은 첫 자전거 종주 돌아오 는 길 에 묵 었 던 충주 자연 호...
239,송정그림마을,여행부여여행 송정그림책마을 보는곳 마다 그림같은 곳 늘봄 복사 이웃추가본...,여행 부여 여행 송정 그림책 마을 보 는 곳 마다 그림 같 은 곳 늘봄 복사 이웃...
383,소래습지생태공원,작년에 다쳤던 발목에 아직도 통증이 있어서 등산을 반년 가까이 쉬고있다지난 년간 거...,작년 에 다쳤 던 발목 에 아직 도 통증 이 있 어서 등산 을 반년 가까이 쉬 고...
430,이기대 (부산 국가지질공원),광역시부산여행 이기대 해안산책로 해안을 따라 걷는 부산국가지질공원 소금성 ...,광역시 부산 여행 이기대 해안 산책로 해안 을 따라 걷 는 부산 국가 지질 공원 ...
233,한라산 (제주도 국가지질공원),여행제주 제주도 국가지질공원 한라산 복사 이웃추가본문 기타 기능번역보기...,여행 제주 제주도 국가 지질 공원 한라산 복사 이웃 추가 본문 기타 기능 번역 보...
439,화진포 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
428,선운산 (전북 서해안권 국가지질공원),여행정보전북여행 서해안권 변산반도 국가지질공원 직소폭포 알쓸신잡 복사 이웃...,여행 정보 전북 여행 서해안 권 변산반도 국가 지질 공원 직소 폭포 알 쓸 신잡 ...
874,무주머루와인동굴,적상산 적상호 주변단풍머루와인동굴 무주 머루와인동굴적상호적상호 전망대적상호 둘레길무...,적상 산 적 상호 주변 단풍 머루 와 인동 굴 무주 머루 와 인동 굴 적 상호 적...
793,고복자연공원,내이야기세종시의 월 고복자연공원과 베어트리파크 여름 복사 이웃추가본문 기...,내 이야기 세 종시 의 월 고복 자연 공원 과 베 어 트리 파크 여름 복사 이웃 ...
269,이천 부래미마을,이천여행경기도 아이와 가볼만한곳 이천 농촌체험마을로 가을 나들이 부래미 마을 편 ...,이 천 여행 경기 도 아이 와 가 볼 만 한 곳 이 천 농촌 체험 마을 로 가을 ...


#### 영화 : 주토피아 결과

In [52]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "가족 아이 동물")
df.iloc[recommand_list]

,NAME,description,token_descirption
961,부천 동물테마파크 하이주,아이와 갈만한곳부천 아이와 갈만한 곳 부천 동물테마파크 하이주 러블리뭉치 ...,아이 와 갈만 한 곳 부천 아이 와 갈만 한 곳 부천 동물 테마파크 하이주 러블리...
287,서울대공원,육아일기서울대공원 아이와 함께 나들이 그레텔 복사 이웃추가본문 기타 기능...,육아 일기 서울대 공원 아이 와 함께 나 들 이 그레 텔 복사 이웃 추가 본문 기...
775,우농타조농장,가볼만한곳파주 가볼만한곳 우농타조농장 타조마을 어린이체험이 가득한 곳 복...,가 볼 만 한 곳 파주 가 볼 만 한 곳 우 농 타조 농장 타조 마을 어린이 체험...
530,화성 베어팜,베어팜 시설 안내화성 베어팜 자연 체험 동물편 베어팜돌이 복사 이웃추가본...,베어 팜 시설 안내 화성 베어 팜 자연 체험 동물 편 베어 팜 돌 이 복사 이웃 ...
96,아산 환경과학공원,충청남도아산여행 그린타워 황금빛 들판이 한눈에 들어오는 아산환경과학공원에 위치한 아...,충청남도 아산 여행 그린 타워 황금빛 들판 이 한눈 에 들어오 는 아산 환경 과학...
843,쇠꼴마을,잡다한 얘기파주 귀한농부 캠핑장 쇠꼴마을 후기 스마트코더 복사 이웃추가본문...,잡다 한 얘기 파주 귀한 농부 캠 핑장 쇠꼴 마을 후기 스마트 코더 복사 이웃 추...
207,돼지문화원,국내여행강원도여행 원주 돼지문화원 아기와 가볼만한 곳 별양 복사 이웃추...,국내 여행 강원 도 여행 원주 돼지 문화원 아기 와 가 볼 만 한 곳 별양 복사 ...
467,리틀빅아이 자유숲놀이터,리틀빅아이 자유숲놀이터지난 주말 미리 사전투표하고오늘은 하마마리와 함께숲놀이터라는 ...,리틀 빅 아이 자유 숲 놀이터 지난 주말 미리 사전 투표 하 고 오늘 은 하마 마...
738,단양 천동동굴,단양 가볼만한 곳 천동동굴 년 월 일 토요일아직도 코로나가 계속 늘어나고 있어...,단양 가 볼 만 한 곳 천 동동 굴 년 월 일 토요일 아직 도 코 로 나가 계속 ...
848,아쿠아플라넷 63,일상다반사가을나들이 아쿠아플라넷에서 인생사진 찍었어요 복사 이웃추가본문...,일상다반사 가 을 나 들 이 아쿠아 플라 넷 에서 인생 사진 찍 었 어요 복사 이...


#### 영화 : 마션 결과

In [50]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "화성 우주 기지 생존 우주선 우주인")
df.iloc[recommand_list]

,NAME,description,token_descirption
715,지리산온천랜드,국내여행겨울여행 추천 지리산 온천랜드 켄싱턴리조트 지리산남원 힐링 온천 패키지 네히...,국내 여행 겨울 여행 추천 지리산 온천 랜드 켄싱 턴 리조트 지리산 남원 힐링 온...
605,산들소리수목원,육아일기산들소리수목원 아이랑 가기 너무 좋아요 뽀블링 복사 이웃추가본문 기...,육아일 기산 들 소리 수목원 아이 랑 가 기 너무 좋 아요 뽀 블링 복사 이웃 추...
837,땅끝관광지,광주전남해남땅끝마을 땅끝관광지 여행과 노화도 보길도 배 시간 공유 실리동효소정후자 ...,광주 전남 해남 땅 끝 마을 땅 끝 관광지 여행 과 노화 도 보 길 도 배 시간 ...
208,제주허브동산,국내여행월 제주도 여행 눈코입 모두 즐거운 제주 허브동산 럭키윰 복사 이...,국내 여행 월 제주 도 여행 눈코 입 모두 즐거운 제주 허브 동산 럭키 윰 복사 ...
19,청초호,속초 여행 봉브레드와 청초호 짬뽕순두부 마이로 복사 이웃추가본문 기타 기...,속초 여행 봉 브레드 와 청초호 짬뽕 순두부 마이 로 복사 이웃 추가 본문 기타 ...
264,제주 산방산탄산온천,노천탕도 있는 제주 산방산탄산온천 마이여니 복사 이웃추가본문 기타 기능지도...,노천탕 도 있 는 제주 산방산 탄산 온천 마이 여니 복사 이웃 추가 본문 기타 기...
433,홍천 배바위카누마을,놀고홍천 여행 홍천 가볼만한곳 배바위카누마을 아이들 체험학습캠핑장 황장금 ...,놀 고 홍천 여행 홍천 가 볼 만 한 곳 배바위 카누 마을 아이 들 체험학습 캠 ...
337,영등포 신길동 홍어거리,살아가는 이야기영등포 신길동 홍어거리 원조 홍어 본점 비오는날 막걸리가 최고 음...,살아가 는 이야기 영등포 신길동 홍어 거리 원조 홍어 본점 비오 는 날 막걸리 가...
277,원당종마목장 (렛츠런팜 원당),경기도고양 가볼만한곳 원당종마목장렛츠런팜 원당 모래성 복사 이웃추가본문 기...,경기도 고양 가 볼 만 한 곳 원당 종마 목장 렛츠 런 팜 원당 모래 성 복사 이...
971,홍원항,국내여행서천 가볼만한곳 홍원항 노지 클린캠핑 낚시와 주변청소 까지 채이파파 ...,국내 여행 서천 가 볼 만 한 곳 홍원항 노지 클린 캠핑 낚시 와 주변 청소 까지...


## 4th. DOC2vec-위키피디아 글뭉치 문단 단위 평균
- 블로그글을 읽을때 사람들은 관심있는 부분을 보는경우가 많다 따라서 문단중 거리가 일정 수준이상 멀어진 글을 제거하고 나머지 문단들의 평균을 값을 활용하여 거리를 구함.
- 문제점 : 봄날은 간다, 주토피아의 경우 좋은결과를 보여주었지만 여전히 마션의경우 좋지 않은 결과를 보여줌

In [39]:
def doc2vec_recommand_list(model, keyword):
    from gensim.models import doc2vec
    from scipy import spatial
    from konlpy.tag import Mecab

    # tokenize
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word

    # vectorize
    model.random.seed(0) # doc2vec은 random값을 고정해주어야 똑같은 벡터값이 출력됨
    fisrt_text = sentence
    vec1 = model.infer_vector(fisrt_text.split())
    similar = []
    for i, row in df.iterrows():
        tenword_split = row.token_descirption.split()
        total_sim = []
        temp = []
        for word in tenword_split:
            temp.append(word)
            
            if len(temp) ==200: # 200은 문장 6줄을 형태소 분석하면 나오 원소들의 갯수
                     
                vec2 = model.infer_vector(temp)
                similairty = spatial.distance.cosine(vec1, vec2)
                if similairty < 0.9:
                    total_sim.append(similairty)
                    
                temp = []
        if len(total_sim) == 0: #전부 연관이 없는 문맥만 있었을 경우
            similar.append(4) # 4라는 값은 임의의 값이다 거리를 매우 크게 주기위하여
            print("나왔다")
        else:
            similar.append(np.mean(total_sim))
    print(len(similar))
    print()  
    return list(np.array(similar).argsort()[:10]) #상위 열개만을 리스트로 반환

#### 영화 : 봄날은 간다 결과

In [41]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "자연 대나무 숲")
df.iloc[recommand_list]

990



,NAME,description,token_descirption
439,화진포 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
82,두타연 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
474,충주자연호텔,안녕하세요 오늘은 첫 자전거 종주 돌아오는 길에 묵었던 충주 자연 호텔 사우나에 대...,안녕 하 세요 오늘 은 첫 자전거 종주 돌아오 는 길 에 묵 었 던 충주 자연 호...
408,송지호 해안 서낭바위 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
761,단양 방곡도깨비마을,체험마을 여행기여 년 전의 도예의 숨결이 살아 숨 쉬는 곳 방곡도깨비마을농촌여행 웰...,체험 마을 여행기 여 년 전 의 도예 의 숨결 이 살 아 숨 쉬 는 곳 방곡 도깨...
151,용천동굴 [유네스코 세계자연유산],관광지 투어제주 용천동굴 유네스코 세계자연유산 용암동굴제주도렌트카협동조합 ...,관광지 투어 제주 용천 동굴 유네스코 세계 자연 유산 용암 동굴 제주도 렌트카 협...
332,쁘띠프랑스,경기도 가평군가평 여행 쁘띠프랑스 복사 이웃추가본문 기타 기능지도로 ...,경기도 가평군 가평 여행 쁘띠프랑스 복사 이웃 추가 본문 기타 기능 지도 로 보 ...
594,안산갈대습지공원,국내여행조류 탐사가 가능한 안산 갈대습지공원 뇽뇽 복사 이웃추가본문 기타 ...,국내 여행 조류 탐사 가 가능 한 안산 갈대 습지 공원 뇽뇽 복사 이웃 추가 본문...
168,서울 동대문 닭한마리 골목,진짜 맛집 혹은 그냥 식당동대문 닭한마리 골목에 가보다 진원조 닭한마리 수채화용구 ...,진짜 맛집 혹은 그냥 식당 동대문 닭 한 마리 골목 에 가 보 다 진 원조 닭 한...
442,검룡소 (강원고생대 국가지질공원),중부내륙 힐링투어 단양 네델란드 풍차마을 영월 강원고생대국가지질공원 칼리 ...,중부 내륙 힐링 투어 단양 네 델 란 드 풍차 마을 영월 강원 고생대 국가 지질 ...


#### 영화 : 주토피아 결과

In [42]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "가족 아이 동물")
df.iloc[recommand_list]

990



,NAME,description,token_descirption
467,리틀빅아이 자유숲놀이터,리틀빅아이 자유숲놀이터지난 주말 미리 사전투표하고오늘은 하마마리와 함께숲놀이터라는 ...,리틀 빅 아이 자유 숲 놀이터 지난 주말 미리 사전 투표 하 고 오늘 은 하마 마...
775,우농타조농장,가볼만한곳파주 가볼만한곳 우농타조농장 타조마을 어린이체험이 가득한 곳 복...,가 볼 만 한 곳 파주 가 볼 만 한 곳 우 농 타조 농장 타조 마을 어린이 체험...
207,돼지문화원,국내여행강원도여행 원주 돼지문화원 아기와 가볼만한 곳 별양 복사 이웃추...,국내 여행 강원 도 여행 원주 돼지 문화원 아기 와 가 볼 만 한 곳 별양 복사 ...
145,두물머리 애벌레 생태학교,양평 두물머리 애벌레 생태학교날이 정말정말 좋은 오늘수요일 교시 하는 날인데ㅠㅠ현장...,양평 두 물머리 애벌레 생태 학교 날 이 정말 정말 좋 은 오늘 수요일 교시 하 ...
288,서울어린이대공원,년코로나 피해 나들이 서울어린이대공원 놀이공원가요 복사 이웃추가본문 기...,년 코 로 나 피해 나 들 이 서울 어린이 대공원 놀이공원 가요 복사 이웃 추가 ...
605,산들소리수목원,육아일기산들소리수목원 아이랑 가기 너무 좋아요 뽀블링 복사 이웃추가본문 기...,육아일 기산 들 소리 수목원 아이 랑 가 기 너무 좋 아요 뽀 블링 복사 이웃 추...
52,덕평공룡수목원,쑥쑥크는 예봄이아기랑 갈만한곳 덕평공룡수목원 간만에 즐거운 나들이 씬 복...,쑥쑥 크 는 예 봄 이 아기 랑 갈만 한 곳 덕평 공룡 수목원 간 만 에 즐거운 ...
961,부천 동물테마파크 하이주,아이와 갈만한곳부천 아이와 갈만한 곳 부천 동물테마파크 하이주 러블리뭉치 ...,아이 와 갈만 한 곳 부천 아이 와 갈만 한 곳 부천 동물 테마파크 하이주 러블리...
96,아산 환경과학공원,충청남도아산여행 그린타워 황금빛 들판이 한눈에 들어오는 아산환경과학공원에 위치한 아...,충청남도 아산 여행 그린 타워 황금빛 들판 이 한눈 에 들어오 는 아산 환경 과학...
530,화성 베어팜,베어팜 시설 안내화성 베어팜 자연 체험 동물편 베어팜돌이 복사 이웃추가본...,베어 팜 시설 안내 화성 베어 팜 자연 체험 동물 편 베어 팜 돌 이 복사 이웃 ...


#### 영화 : 마션 결과

In [43]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "화성 우주 기지 생존 우주선 우주인")
df.iloc[recommand_list]

990



,NAME,description,token_descirption
761,단양 방곡도깨비마을,체험마을 여행기여 년 전의 도예의 숨결이 살아 숨 쉬는 곳 방곡도깨비마을농촌여행 웰...,체험 마을 여행기 여 년 전 의 도예 의 숨결 이 살 아 숨 쉬 는 곳 방곡 도깨...
498,거제 학동리 동백나무 숲 및 팔색조 번식지,거제 학동리 동백나무 숲 및 팔색조번식지에 대한 설명입니다동백나무는 차나무과에 속하...,거제 학동리 동백나무 숲 및 팔색조 번식지 에 대한 설명 입니다 동백나무 는 차 ...
423,대흥사[유네스코 세계문화유산],해남 대흥사유네스코 세계 문화유산 대 사찰 여행 해남 대흥사 낭만충전 복사...,해남 대흥사 유네스코 세계 문화유산 대 사찰 여행 해남 대흥사 낭만 충전 복사 이...
332,쁘띠프랑스,경기도 가평군가평 여행 쁘띠프랑스 복사 이웃추가본문 기타 기능지도로 ...,경기도 가평군 가평 여행 쁘띠프랑스 복사 이웃 추가 본문 기타 기능 지도 로 보 ...
408,송지호 해안 서낭바위 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
467,리틀빅아이 자유숲놀이터,리틀빅아이 자유숲놀이터지난 주말 미리 사전투표하고오늘은 하마마리와 함께숲놀이터라는 ...,리틀 빅 아이 자유 숲 놀이터 지난 주말 미리 사전 투표 하 고 오늘 은 하마 마...
541,부산 감천문화마을,여행 스케치부산 감천문화마을 볼거리 많아 걷기 좋은곳 대니얼과 제이미 복사...,여행 스케치 부산 감천 문화 마을 볼거리 많 아 걷 기 좋 은 곳 대니얼 과 제 ...
735,예천 출렁다리마을,경북 추천여행예천 출렁다리마을과 고택 초간정 경북예천여행지 예천출렁다리 예천초간정...,경북 추천 여행 예천 출렁다리 마을 과 고택 초 간정 경북 예천 여행지 예천 출렁...
439,화진포 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
412,"여주 영릉(英陵)세종대왕)과 영릉(寧陵)효종,인선왕후) [유네스코 세계문화유산]",문화경기 여주 유네스코 세계문화유산여주 영릉세종대왕과 영릉효종인선왕후 ...,문화 경기 여주 유네스코 세계 문화유산 여주 영릉 세종 대왕 과 영릉 효종 인 선...


## 5th. DOC2vec-위키피디아 글뭉치 문단 단위 평균

In [ ]:
from gensim.models import doc2vec
import sys
import multiprocessing

cores = multiprocessing.cpu_count()

# doc2vec parameters 

vector_size = 300
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 10

dm = 0  # {0:dbow, 1:dmpv}

worker_count = cores  # number of parallel processes
ROOT_DIR = "C:\\code\\"
inputfile = ROOT_DIR + "wiki_pos_tokenizer_without_taginfo.txt" 
modelfile = ROOT_DIR + "wiki_pos_tokenizer_without_taginfo_pvdbow.doc2vec.model"
word2vec_file = modelfile + ".word2vec_format"

sentences = doc2vec.TaggedLineDocument(inputfile)

# build voca

model = doc2vec.Doc2Vec(min_count=word_min_count, vector_size=vector_size,

                        alpha=0.025, min_alpha=0.025, seed=1234, workers=worker_count)

model.build_vocab(sentences)

# Train document vectors

model.train(sentences, epochs=model.iter, total_examples=model.corpus_count)
# To save
model.save(modelfile)
model.save_word2vec_format(word2vec_file, binary=False)

In [34]:
from gensim.models import doc2vec
doc2vec_model = doc2vec.Doc2Vec.load("./wiki_pos_tokenizer_without_taginfo_pvdbow.doc2vec.model")

#### 영화 : 봄날은 간다 결과

In [35]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "자연 대나무 숲")
df.iloc[recommand_list]

,NAME,description,token_descirption
474,충주자연호텔,안녕하세요 오늘은 첫 자전거 종주 돌아오는 길에 묵었던 충주 자연 호텔 사우나에 대...,안녕 하 세요 오늘 은 첫 자전거 종주 돌아오 는 길 에 묵 었 던 충주 자연 호...
310,부소담악(추소정),옥천을 가본지가 엄청 오래되어서한번 여행 가보려고 네이버에서 검색결과 부소담악이 눈...,옥천 을 가 본지 가 엄청 오래 되 어서 한 번 여행 가 보 려고 네이버 에서 검...
530,화성 베어팜,베어팜 시설 안내화성 베어팜 자연 체험 동물편 베어팜돌이 복사 이웃추가본...,베어 팜 시설 안내 화성 베어 팜 자연 체험 동물 편 베어 팜 돌 이 복사 이웃 ...
439,화진포 (강원평화지역 국가지질공원),백패킹 강원도 고성 가볼만한곳 송지호 해안 서낭바위 강원평화지역 국가지질공원 알마 ...,백 패킹 강원도 고성 가 볼 만 한 곳 송지호 해안 서낭 바위 강원 평화 지역 국...
423,대흥사[유네스코 세계문화유산],해남 대흥사유네스코 세계 문화유산 대 사찰 여행 해남 대흥사 낭만충전 복사...,해남 대흥사 유네스코 세계 문화유산 대 사찰 여행 해남 대흥사 낭만 충전 복사 이...
122,"금강자연휴양림(금강수목원,산림박물관)",세종시 정보세종시 금강수목원 금강자연휴양림 산림 박물관세종랜드 부동산 복사...,세 종시 정보 세 종시 금강 수목원 금강 자연 휴양림 산림 박물관 세종 랜드 부동...
322,경상남도수목원,여행경상남도 진주 반성 수목원 나들이 가기 정말 좋은 이유 웅이일상 복사 ...,여행 경상 남 도 진주 반성 수목원 나 들 이 가 기 정말 좋 은 이유 웅 이 일...
594,안산갈대습지공원,국내여행조류 탐사가 가능한 안산 갈대습지공원 뇽뇽 복사 이웃추가본문 기타 ...,국내 여행 조류 탐사 가 가능 한 안산 갈대 습지 공원 뇽뇽 복사 이웃 추가 본문...
70,국립 운장산자연휴양림,월 봄꽃 만발한 외갓집 같은 캠핑장국립운장산자연휴양림 캠핑장글 한형석 친환경캠핑스...,월 봄꽃 만발 한 외갓집 같 은 캠핑 장국립 운장산 자연 휴양림 캠 핑장 글 한 ...
777,청풍호 관광모노레일,제목은 청풍호 둘러보기인데 사실 처음 의도는 이게 아니었다주말에 단양의 만천하스카이...,제목 은 청풍 호 둘러보 기 인데 사실 처음 의도 는 이게 아니 었 다 주말 에 ...


#### 영화 : 주토피아 결과

In [36]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "가족 아이 동물")
df.iloc[recommand_list]

,NAME,description,token_descirption
961,부천 동물테마파크 하이주,아이와 갈만한곳부천 아이와 갈만한 곳 부천 동물테마파크 하이주 러블리뭉치 ...,아이 와 갈만 한 곳 부천 아이 와 갈만 한 곳 부천 동물 테마파크 하이주 러블리...
96,아산 환경과학공원,충청남도아산여행 그린타워 황금빛 들판이 한눈에 들어오는 아산환경과학공원에 위치한 아...,충청남도 아산 여행 그린 타워 황금빛 들판 이 한눈 에 들어오 는 아산 환경 과학...
54,베어트리파크,국내여행서울 근교 봄나들이 세종 베어트리파크 네히르 복사 이웃추가본문 기타...,국내 여행 서울 근교 봄 나 들 이 세종 베어 트리 파크 네히 르 복사 이웃 추가...
145,두물머리 애벌레 생태학교,양평 두물머리 애벌레 생태학교날이 정말정말 좋은 오늘수요일 교시 하는 날인데ㅠㅠ현장...,양평 두 물머리 애벌레 생태 학교 날 이 정말 정말 좋 은 오늘 수요일 교시 하 ...
288,서울어린이대공원,년코로나 피해 나들이 서울어린이대공원 놀이공원가요 복사 이웃추가본문 기...,년 코 로 나 피해 나 들 이 서울 어린이 대공원 놀이공원 가요 복사 이웃 추가 ...
287,서울대공원,육아일기서울대공원 아이와 함께 나들이 그레텔 복사 이웃추가본문 기타 기능...,육아 일기 서울대 공원 아이 와 함께 나 들 이 그레 텔 복사 이웃 추가 본문 기...
775,우농타조농장,가볼만한곳파주 가볼만한곳 우농타조농장 타조마을 어린이체험이 가득한 곳 복...,가 볼 만 한 곳 파주 가 볼 만 한 곳 우 농 타조 농장 타조 마을 어린이 체험...
848,아쿠아플라넷 63,일상다반사가을나들이 아쿠아플라넷에서 인생사진 찍었어요 복사 이웃추가본문...,일상다반사 가 을 나 들 이 아쿠아 플라 넷 에서 인생 사진 찍 었 어요 복사 이...
843,쇠꼴마을,잡다한 얘기파주 귀한농부 캠핑장 쇠꼴마을 후기 스마트코더 복사 이웃추가본문...,잡다 한 얘기 파주 귀한 농부 캠 핑장 쇠꼴 마을 후기 스마트 코더 복사 이웃 추...
181,송암스페이스센터,우리나라에서 일본의 영향을 받아 혈액형으로 사람을 구분하듯이서양에서는 별자리로 사람...,우리 나라 에서 일본 의 영향 을 받 아 혈액형 으로 사람 을 구분 하 듯이 서양...


#### 영화 : 마션 결과

In [37]:
recommand_list = doc2vec_recommand_list(doc2vec_model, "화성 우주 기지 생존 우주선 우주인")
df.iloc[recommand_list]

,NAME,description,token_descirption
707,춘천 옥산가(옥광산),춘천 옥산가 옥광산 찜질방 살아기도 좋아해요안녕하세요최악의 미세먼지로 답답한 월이네...,춘천 옥산 가 옥 광산 찜질방 살 아기 도 좋 아 해요 안녕 하 세요 최악 의 미...
538,두륜산케이블카,여행맛집부모님과 떠나는 전라도 별미 웰빙 박 일 여행후기두륜산케이블카두륜산힐링로드가...,여행 맛집 부모 님 과 떠나 는 전라도 별미 웰빙 박 일 여행 후기 두륜산 케이블...
420,흥정계곡,놀러가기강원도 평창 흥정계곡 계획없이 떠나기초이린 복사 이웃추가본문 기타 ...,놀 러 가 기 강원도 평창 흥정 계곡 계획 없이 떠나 기초 이 린 복사 이웃 추가...
320,사북 탄광문화관광촌,강원도정선 여행 가볼 만한 곳 사북 탄광문화관광촌 갱도 체험 보우 복사 ...,강원 도정선 여행 가 볼 만 한 곳 사북 탄 광문 화관 광촌 갱 도 체험 보우 복...
814,목포 관광특구,팔팔장수회와 행복 동아리목포 유달산공원 외에 해양 문화 특구 관광허니 복...,팔 팔 장 수회 와 행복 동아리 목포 유달산 공원 외 에 해양 문화 특구 관광 허...
165,한려수도 조망케이블카,경상도통영 거제 여행 한려수도 조망 케이블카 박경리 기념관 역마살 복사 ...,경상도 통영 거제 여행 한 려수 도 조망 케이블카 박경리 기념관 역마살 복사 이웃...
604,땅끝모노레일,전라도해남 땅끝전망대 땅끝 모노레일로 다녀왔어요 행복한여행 복사 이웃추가본...,전라도 해남 땅 끝 전망대 땅 끝 모노레일 로 다녀왔 어요 행복 한 여행 복사 이...
837,땅끝관광지,광주전남해남땅끝마을 땅끝관광지 여행과 노화도 보길도 배 시간 공유 실리동효소정후자 ...,광주 전남 해남 땅 끝 마을 땅 끝 관광지 여행 과 노화 도 보 길 도 배 시간 ...
455,대관령 관광특구,여행강원 평창 강원도의 산악 지대와 해안을 아우르는 대관령 관광특구 ...,여행 강원 평창 강원도 의 산악 지대 와 해안 을 아우르 는 대관령 관광특구 복사...
663,성불계곡,무한한 먹방광양계곡추천성불계곡 수빈 복사 이웃추가본문 기타 기능번역보기안...,무한 한 먹방 광양 계 곡추 천성 불계 곡 수빈 복사 이웃 추가 본문 기타 기능 ...


# LSA
- 앞서 만들어 두었던 TFIDF행렬에 SVD로 축소행렬을 만들어 줌으로써 연산량을 줄이며 위에서 발견하지 못하였던 잠재적의미를 추출함을 목표로함.
- 100, 200, 300, 차원등 차원을 늘리는 작업을 시도해 보았으나 특별한 변화가 보이지 않아 100차원으로 선정함.
- 추천의 결과 대게 좋은 결과를 보임

In [3]:
def tfidf_lsa_model(keyword):
    
    from konlpy.tag import Mecab
    from sklearn.decomposition import TruncatedSVD
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.feature_extraction.text import TfidfVectorizer
    import scipy as sp

    vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')
    blog_contents = list(df.token_descirption)
    X = vectorizer.fit_transform(blog_contents)
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word
    sentence_ls = [sentence]
    
    def dist_norm(v1, v2):
        v1_normalized = v1 / sp.linalg.norm(v1)
        v2_normalized = v2 / sp.linalg.norm(v2)

        delta = v1_normalized  - v2_normalized

        return sp.linalg.norm(delta)


    svd = TruncatedSVD(n_components=100)
    vecs = svd.fit_transform(X)
    sentence_ls
    test_vec = vectorizer.transform(sentence_ls)
    test_vec = svd.transform(test_vec)
    dist = [dist_norm(each, test_vec) for each in vecs]
    dist_arr = np.array([dist])
    find_ind = dist_arr.argsort()
    new_ind = list(find_ind[:, :10][0])
    return new_ind

#### 영화 : 봄날은 간다 결과

In [4]:
tfidf_lsa_model_list = tfidf_lsa_model("자연 대나무 숲")
df.iloc[tfidf_lsa_model_list]

,NAME,description,token_descirption
284,대나무골 테마공원,국내여행담양 대나무골 테마공원 푸른 대나무의 향기메타세콰이아가로수길 소쇄원 ...,국내 여행담 양 대나무 골 테마 공원 푸른 대나무 의 향기 메타 세 콰 이아 가로...
884,담양 소쇄원,전라남도 담양군월의 담양 여행 소쇄원과 광주호 호수생태원 복사 이웃추...,전라남도 담양군 월 의 담양 여행 소쇄원 과 광주호 호수 생태 원 복사 이웃 추가...
684,담양 명옥헌 원림,전라도담양 명옥헌 원림 배롱나무의 아름다움 속으로 일상탈출 복사 이웃추가본...,전라도 담양 명 옥 헌 원림 배롱나무 의 아름다움 속 으로 일상 탈출 복사 이웃 ...
772,죽녹원,전라남도 담양군담양 죽녹원과 담양호 용마루길 복사 이웃추가본문 기타 기...,전라남도 담양군 담양 죽 녹원 과 담양호 용마루 길 복사 이웃 추가 본문 기타 기...
679,담양 메타세쿼이아길,그 동안 코로나로 방콕만 한 우리 아이들을 위해어린이날은 어디라도 가야지코로나로 인...,그 동안 코로나 로 방콕 만 한 우리 아이 들 을 위해 어린이 날 은 어디 라도 ...
550,전남 담양 창평 [슬로시티],추천여행지전남 담양 창평 슬로시티에서 담빛 예술창고까지 담양 추천 여행지 김경기 ...,추천 여행지 전남 담양 창평 슬로 시티 에서 담 빛 예술 창고 까지 담양 추천 여...
531,아홉산숲,부산 살면서 가보고 싶던 아홉산숲을 마침내 다녀왔다 예약을 해야 하고 제한된 인원만...,부산 살 면서 가 보 고 싶 던 아홉 산 숲 을 마침내 다녀왔 다 예약 을 해야 ...
234,태화강 국가정원,국내여행울산 가볼만한곳 태화강 국가정원 양귀비 금영화 라벤더 따스한 봄 이매미 ...,국내 여행 울산 가 볼 만 한 곳 태화강 국가 정원 양귀비 금영화 라벤더 따스 한...
313,태화강,국내여행울산 가볼만한곳 태화강 국가정원 양귀비 금영화 라벤더 따스한 봄 이매미 ...,국내 여행 울산 가 볼 만 한 곳 태화강 국가 정원 양귀비 금영화 라벤더 따스 한...
859,광주호 호수생태원,전라남도 담양군월의 담양 여행 소쇄원과 광주호 호수생태원 복사 이웃추...,전라남도 담양군 월 의 담양 여행 소쇄원 과 광주호 호수 생태 원 복사 이웃 추가...


#### 영화 : 주토피아 결과

In [5]:
tfidf_lsa_model_list = tfidf_lsa_model("가족 아이 동물")
df.iloc[tfidf_lsa_model_list]

,NAME,description,token_descirption
961,부천 동물테마파크 하이주,아이와 갈만한곳부천 아이와 갈만한 곳 부천 동물테마파크 하이주 러블리뭉치 ...,아이 와 갈만 한 곳 부천 아이 와 갈만 한 곳 부천 동물 테마파크 하이주 러블리...
775,우농타조농장,가볼만한곳파주 가볼만한곳 우농타조농장 타조마을 어린이체험이 가득한 곳 복...,가 볼 만 한 곳 파주 가 볼 만 한 곳 우 농 타조 농장 타조 마을 어린이 체험...
207,돼지문화원,국내여행강원도여행 원주 돼지문화원 아기와 가볼만한 곳 별양 복사 이웃추...,국내 여행 강원 도 여행 원주 돼지 문화원 아기 와 가 볼 만 한 곳 별양 복사 ...
467,리틀빅아이 자유숲놀이터,리틀빅아이 자유숲놀이터지난 주말 미리 사전투표하고오늘은 하마마리와 함께숲놀이터라는 ...,리틀 빅 아이 자유 숲 놀이터 지난 주말 미리 사전 투표 하 고 오늘 은 하마 마...
170,알파카월드,국내홍천 아이랑 가볼만한곳 홍천 알파카월드 딘듀봉 복사 이웃추가본문 기타 ...,국내 홍천 아이 랑 가 볼 만 한 곳 홍천 알파카 월드 딘 듀 봉 복사 이웃 추가...
54,베어트리파크,국내여행서울 근교 봄나들이 세종 베어트리파크 네히르 복사 이웃추가본문 기타...,국내 여행 서울 근교 봄 나 들 이 세종 베어 트리 파크 네히 르 복사 이웃 추가...
530,화성 베어팜,베어팜 시설 안내화성 베어팜 자연 체험 동물편 베어팜돌이 복사 이웃추가본...,베어 팜 시설 안내 화성 베어 팜 자연 체험 동물 편 베어 팜 돌 이 복사 이웃 ...
53,안성팜랜드,알뜰정보안성팜랜드 요즘 상황 답답해서 댕겨옴 쪼꼬 복사 이웃추가본문 기타...,알뜰 정보 안성 팜 랜드 요즘 상황 답답 해서 댕겨 옴 쪼 꼬 복사 이웃 추가 본...
229,대전오월드,제주말고대전 오월드에서 겹벚꽃 튤립 보고왔어용 알린 복사 이웃추가본문 기...,제주 말 고대 전 오 월드 에서 겹벚꽃 튤립 보 고 왔 어용 알린 복사 이웃 추가...
145,두물머리 애벌레 생태학교,양평 두물머리 애벌레 생태학교날이 정말정말 좋은 오늘수요일 교시 하는 날인데ㅠㅠ현장...,양평 두 물머리 애벌레 생태 학교 날 이 정말 정말 좋 은 오늘 수요일 교시 하 ...


#### 결과 : 마션 결과

In [6]:
tfidf_lsa_model_list = tfidf_lsa_model("화성 우주 기지 생존 우주선 우주인")
df.iloc[tfidf_lsa_model_list]

,NAME,description,token_descirption
65,나로우주센터 우주과학관,년도 여행고흥여행 나로우주센터 우주과학관현답 복사 이웃추가본문 기타 기능번...,년 도 여행 고흥 여행 나 로 우주 센터 우주 과학관 현답 복사 이웃 추가 본문 ...
736,옥토끼우주센터,전국공지 강화도 옥토끼우주센터 솔직한 방문 후기 친절한 유박사 복사 이웃추...,전국 공지 강화 도 옥토끼 우주 센터 솔직 한 방문 후기 친절 한 유박 사 복사 ...
485,사천 항공우주박물관,생일이 같은 남매이야기사천 항공우주박물관 남해여행 아이랑 가볼만한곳 쌀몽맘 ...,생일 이 같 은 남매 이야기 사 천 항공 우주 박물관 남 해 여행 아이 랑 가 볼...
558,남열해돋이해수욕장,국내여행고성 남열해돋이해수욕장낭만서프하우스 원룸형 펜션숙소박 후기 프레시 ...,국내 여행 고성 남 열 해돋이 해수욕장 낭 만 서프 하우스 원룸 형 펜션 숙소 박...
51,별빛정원우주,아이와 함께 가기 좋은 곳서울근교 가볼만한 곳 이천 별빛정원 우주 항상웃는 ...,아이 와 함께 가 기 좋 은 곳 서울 근교 가 볼 만 한 곳 이 천 별빛 정원 우...
808,고흥 유자공원,고흥을 만나다고흥 유자공원 유자 보러 공원으로 가보자 고흥관광 복사 이웃추...,고흥 을 만나 다고 흥 유자 공원 유자 보 러 공원 으로 가 보 자 고흥 관광 복...
844,수원 화성 [유네스코 세계문화유산],수원화성유네스코 세계문화유산 수원화성 한 바퀴대교약졸 복사 이웃추가본문 기...,수원 화성 유네스코 세계 문화유산 수원 화성 한 바퀴 대교약졸 복사 이웃 추가 본...
853,방화수류정(동북각루),일상년 월 세번째카트라이더 스플랜더 카페비은 오메가 아이브라이트 광교산 형제봉 방화...,일 상년 월 세 번 째 카트라이더 스 플랜 더 카페 비 은 오메가 아이 브라이트 ...
468,화성행궁,블로그서포터즈화성행궁에 봄이 왔대요 수원시청 복사 이웃추가본문 기타 기...,블로그 서포터 즈 화성행궁 에 봄 이 왔 대요 수원 시청 복사 이웃 추가 본문 기...
505,힐링파크 쑥섬쑥섬 (애도),국내여행고흥가볼만한섬쑥섬쑥섬애도힐링파크전남호민간정원꿈길 애니 복사 이웃추가...,국내 여행 고흥 가 볼 만 한 섬쑥 섬쑥 섬 애 도 힐링 파크 전 남호 민간 정원...


# LDA
- 토픽모델링으로 현재 단계에서 여행지 설명글에 토픽은 1개라고 가정하고 토픽을 구성하는 단어의 갯수는 20개만을 추출하여 거리를 구하여 보았다.
- 토픽의 갯수를 늘려보는 방식으로 해당 모델을 튜닝 할 수 있을것으로 판단됨.

In [54]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")

In [55]:
from gensim.models import ldamulticore
from gensim import corpora
def extract_topic(decrition):
    tokenized_corpus = [mecab.nouns(decrition)]
    dictionary = corpora.Dictionary(tokenized_corpus)
    corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]
   
    LDA = ldamulticore.LdaMulticore(corpus, id2word=dictionary, num_topics=1, workers=4)
    all_topics = LDA.get_document_topics(corpus,
                                        minimum_probability=0,
                                        per_word_topics=False)

    topics = LDA.print_topics(num_words=20, num_topics=1)
    main_topic = " ".join(re.findall('[가-힣]+', topics[0][1]))
    return main_topic

In [56]:
df["topic"] = "1"

In [ ]:
for i, row in df.iterrows():
    row['topic'] = extract_topic(row['description'])

In [59]:
def doc2vec_topic_recommand_list(model, keyword):
    from gensim.models import doc2vec
    from scipy import spatial
    from konlpy.tag import Mecab

    # tokenize
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    contents_tokens = mecab.morphs(keyword)
    sentence = ''
    for word in contents_tokens:
        sentence = sentence+' '+word

    # vectorize
    model.random.seed(0)
    fisrt_text = sentence
    vec1 = model.infer_vector(fisrt_text.split())
    similar = []
    for i, row in df.iterrows():
        vec2 = model.infer_vector(row.topic.split())
        similairty = spatial.distance.cosine(vec1, vec2)
        similar.append(similairty)
    print(similar[803])
    return list(np.array(similar).argsort()[:10])

In [237]:
df.to_csv('extracted_topic.csv', encoding='utf-8', index=False)

In [62]:
df = pd.read_csv('./extracted_topic.csv')

#### 영화 : 봄날은 간다 결과

In [63]:
recommand_list = doc2vec_topic_recommand_list(doc2vec_model, "자연 대나무 숲")
df.iloc[recommand_list]

0.7123155295848846


,NAME,description,token_descirption,topic
48,포천 허브아일랜드,타지역맛집더할나위없던 포천 허브아일랜드 맛집 알쏭달쏭 복사 이웃추가본문 기...,타 지역 맛집 더 할 나위 없 던 포천 허브 아일랜드 맛집 알쏭달쏭 복사 이웃 추...,허브 아일랜드 포천 수 곳 것 불빛 축제 꽃 이곳 동화 생각 때 등 여행 전 집 듯...
305,김유정 문학촌,강원도춘천 김유정문학촌 가을 국내 여행지로 추천할만 하죠 공주의아빠 복사 ...,강원도 춘천 김유정 문학 촌 가을 국내 여행지 로 추천 할 만 하 죠 공주 의 아...,김유정 문학 수 촌 춘천 것 곳 작품 작가 이야기 소설 마을 만 여행 년 이곳 생가...
288,서울어린이대공원,년코로나 피해 나들이 서울어린이대공원 놀이공원가요 복사 이웃추가본문 기...,년 코 로 나 피해 나 들 이 서울 어린이 대공원 놀이공원 가요 복사 이웃 추가 ...,어린이 서울 공원 대공원 동물원 동물 수 열차 것 녀석 아이 시간 나 사진 중 곳 ...
498,거제 학동리 동백나무 숲 및 팔색조 번식지,거제 학동리 동백나무 숲 및 팔색조번식지에 대한 설명입니다동백나무는 차나무과에 속하...,거제 학동리 동백나무 숲 및 팔색조 번식지 에 대한 설명 입니다 동백나무 는 차 ...,팔색조 번식지 숲 거제 동백나무 학동 번식 천연기념물 학동리 동백 마리 우리 년 지...
117,북촌 8경,서울삼청동 북촌 북촌 경 풍년쌀농산 츄로 르미름이 복사 이웃추가본문 기...,서울 삼청동 북촌 북촌 경 풍년 쌀 농산 츄 로 르 미름 이 복사 이웃 추가 본문...,북촌 경 길 한옥 곳 마을 북 촌경 가회동 서울 수 지도 골목길 것 풍경 창덕궁 번...
705,기아자동차 화성공장,토지화성 노진리 기아자동차 인근 공장부지 소개합니다 한올공인중개사사무소 복...,토지 화성 노진리 기아 자동차 인근 공장 부지 소개 합니다 한올 공인 중개사 사무...,공장 화성 자동차 기아 매매 분양 지역 시화 반월 공단 스마트 투자 수 평 임대 부...
536,도동서원 [유네스코 세계문화유산],꽃보다 아름다운 년유네스코 세계문화유산도동서원 바람흔적 복사 이웃추가본문 ...,꽃 보다 아름다운 년 유네스코 세계 문화유산 도동 서원 바람 흔적 복사 이웃 추가...,서원 도동 곳 중 년 세계 문화유산 달성 유네스코 것 수 병산 사당 문 여행 강당 ...
744,순천만 용산전망대 (S자 물길),순천여행 순천만습지 자물길 용산전망대 순천 더소풍펜션 순천드라마촬영장 시...,순천 여행 순천 만 습지 자 물 길 용산 전망대 순천 더 소풍 펜션 순천 드라마 ...,순천 전망대 용산 사진 수 일몰 여행 전남 것 시간 곳 물길 길 모습 순천만 습지 ...
511,호도,여행남해여행 남해펜션 호도 섬여행펜션 복사 이웃추가본문 기타 기능번역보...,여행 남 해 여행 남 해 펜션 호도 섬 여행 펜션 복사 이웃 추가 본문 기타 기능...,호도 코코 호두과자 것 답례품 주문 때 수 과자 도 선물 게 우리 맛 개 가능 앙금...
516,부산영화촬영스튜디오,부산영상위원회서포터즈영화로운영화에 관심 있다면 부산영화촬영스튜디오 복사 ...,부산 영상 위원회 서포터 즈 영화 로운 영화 에 관심 있 다면 부산 영화 촬영 스...,부산 영화 촬영 스튜디오 영상 수 위원회 교육 지도 견학 것 사진 컴퓨터 렌탈 작업...


#### 영화 : 주토피아 결과

In [64]:
recommand_list = doc2vec_topic_recommand_list(doc2vec_model, "가족 아이 동물")
df.iloc[recommand_list]

0.8372147679328918


,NAME,description,token_descirption,topic
953,조랑말체험공원,제주도울릉도제주 조랑말체험공원 노란 봄꽃물결 초록바다 최양훈 복사 이웃추가...,제주도 울릉도 제주 조랑말 체험 공원 노란 봄 꽃물결 초록 바다 최 양훈 복사 이...,체험 조랑말 말 승마 공원 수 제주 코스 사진 제주도 곳 유채 꽃 것 거 여행 사람...
143,에버랜드,국내 여행에버랜드 알차게 즐기기 코로나 에버랜드 상황 아가양 복사 이웃추가...,국내 여행 에버 랜드 알차 게 즐기 기 코로나 에버랜드 상황 아 가양 복사 이웃 ...,에버랜드 수 것 용인 호텔 거 아이 동물 주차장 때 곳 사람 분 이용 시간 아들 주...
59,아그로랜드 태신목장,충 남당진 아그로랜드 태신목장 날씨가 흐려도 예쁘네요 롱초 복사 이웃추가본...,충 남당 진 아그 로 랜드 태 신 목장 날씨 가 흐려 도 예쁘 네요 롱 초 복사 ...,목장 곳 랜드 아그 당진 체험 사진 수 아이 것 길 꽃 거 보리밭 트랙터 여행 동물...
145,두물머리 애벌레 생태학교,양평 두물머리 애벌레 생태학교날이 정말정말 좋은 오늘수요일 교시 하는 날인데ㅠㅠ현장...,양평 두 물머리 애벌레 생태 학교 날 이 정말 정말 좋 은 오늘 수요일 교시 하 ...,체험 생태 동물 아이 애벌레 학교 물머리 수 것 등 양평 곤충 램 아들 곳 우리 미...
775,우농타조농장,가볼만한곳파주 가볼만한곳 우농타조농장 타조마을 어린이체험이 가득한 곳 복...,가 볼 만 한 곳 파주 가 볼 만 한 곳 우 농 타조 농장 타조 마을 어린이 체험...,타조 아이 농장 수 동물 곳 농 체험 먹이 것 파주 우 만 토끼 원 마을 거 라면 ...
612,경기도립 물향기수목원,경기 수도권오산 경기도립 물향기수목원 수원 인근 나들이수연수민파파 복사 이...,경기 수도 권 오산 경기 도립 물 향기 수목원 수원 인근 나들이 수연 수민 파파 ...,수목원 물 향기 수 곳 경기도 원 나무 꽃 립 오산 아이 온실 이곳 식물원 가을 여...
170,알파카월드,국내홍천 아이랑 가볼만한곳 홍천 알파카월드 딘듀봉 복사 이웃추가본문 기타 ...,국내 홍천 아이 랑 가 볼 만 한 곳 홍천 알파카 월드 딘 듀 봉 복사 이웃 추가...,알파카 수 월드 곳 먹이 거 산책 동물 홍천 기차 체험 강원도 아이 사진 원 여기 ...
347,퍼스트가든,나들이파주 자동차극장 가볼만한곳 퍼스트가든 반했다 김보송이 복사 이웃추가...,나 들 이 파주 자동차극장 가 볼 만 한 곳 퍼스트 가 든 반 했 다 김 보 송 ...,퍼스트 아이 것 수 사진 파주 곳 체험 가든 시간 축제 지도 저희 동물 만 친구 정...
47,파주 임진각(평화누리공원),싱그러운 계절 파주 가볼만한곳 임진각 평화누리공원 맛찾사 복사 이웃추가본문...,싱그러운 계절 파주 가 볼 만 한 곳 임진 각 평화 누리 공원 맛 찾 사 복사 이...,평화 임진 공원 누리 수 것 곳 파주 아이 사람 사진 거 길 바람 쪽 주차장 전망대...
288,서울어린이대공원,년코로나 피해 나들이 서울어린이대공원 놀이공원가요 복사 이웃추가본문 기...,년 코 로 나 피해 나 들 이 서울 어린이 대공원 놀이공원 가요 복사 이웃 추가 ...,어린이 서울 공원 대공원 동물원 동물 수 열차 것 녀석 아이 시간 나 사진 중 곳 ...


#### 영화 : 마션 결과

In [66]:
recommand_list = doc2vec_topic_recommand_list(doc2vec_model, "화성 우주 기지 생존 우주선 우주인")
df.iloc[recommand_list]

0.6644219160079956


,NAME,description,token_descirption,topic
305,김유정 문학촌,강원도춘천 김유정문학촌 가을 국내 여행지로 추천할만 하죠 공주의아빠 복사 ...,강원도 춘천 김유정 문학 촌 가을 국내 여행지 로 추천 할 만 하 죠 공주 의 아...,김유정 문학 수 촌 춘천 것 곳 작품 작가 이야기 소설 마을 만 여행 년 이곳 생가...
48,포천 허브아일랜드,타지역맛집더할나위없던 포천 허브아일랜드 맛집 알쏭달쏭 복사 이웃추가본문 기...,타 지역 맛집 더 할 나위 없 던 포천 허브 아일랜드 맛집 알쏭달쏭 복사 이웃 추...,허브 아일랜드 포천 수 곳 것 불빛 축제 꽃 이곳 동화 생각 때 등 여행 전 집 듯...
498,거제 학동리 동백나무 숲 및 팔색조 번식지,거제 학동리 동백나무 숲 및 팔색조번식지에 대한 설명입니다동백나무는 차나무과에 속하...,거제 학동리 동백나무 숲 및 팔색조 번식지 에 대한 설명 입니다 동백나무 는 차 ...,팔색조 번식지 숲 거제 동백나무 학동 번식 천연기념물 학동리 동백 마리 우리 년 지...
288,서울어린이대공원,년코로나 피해 나들이 서울어린이대공원 놀이공원가요 복사 이웃추가본문 기...,년 코 로 나 피해 나 들 이 서울 어린이 대공원 놀이공원 가요 복사 이웃 추가 ...,어린이 서울 공원 대공원 동물원 동물 수 열차 것 녀석 아이 시간 나 사진 중 곳 ...
117,북촌 8경,서울삼청동 북촌 북촌 경 풍년쌀농산 츄로 르미름이 복사 이웃추가본문 기...,서울 삼청동 북촌 북촌 경 풍년 쌀 농산 츄 로 르 미름 이 복사 이웃 추가 본문...,북촌 경 길 한옥 곳 마을 북 촌경 가회동 서울 수 지도 골목길 것 풍경 창덕궁 번...
536,도동서원 [유네스코 세계문화유산],꽃보다 아름다운 년유네스코 세계문화유산도동서원 바람흔적 복사 이웃추가본문 ...,꽃 보다 아름다운 년 유네스코 세계 문화유산 도동 서원 바람 흔적 복사 이웃 추가...,서원 도동 곳 중 년 세계 문화유산 달성 유네스코 것 수 병산 사당 문 여행 강당 ...
705,기아자동차 화성공장,토지화성 노진리 기아자동차 인근 공장부지 소개합니다 한올공인중개사사무소 복...,토지 화성 노진리 기아 자동차 인근 공장 부지 소개 합니다 한올 공인 중개사 사무...,공장 화성 자동차 기아 매매 분양 지역 시화 반월 공단 스마트 투자 수 평 임대 부...
511,호도,여행남해여행 남해펜션 호도 섬여행펜션 복사 이웃추가본문 기타 기능번역보...,여행 남 해 여행 남 해 펜션 호도 섬 여행 펜션 복사 이웃 추가 본문 기타 기능...,호도 코코 호두과자 것 답례품 주문 때 수 과자 도 선물 게 우리 맛 개 가능 앙금...
5,도예공방 들꽃마을,들꽃마을 소개도예공방들꽃마을 복사 이웃추가본문 기타 기능번역보기저희공방에...,들꽃 마을 소개 도 예공 방 들꽃 마을 복사 이웃 추가 본문 기타 기능 번역 보 ...,체험 도자기 마을 들꽃 공방 도예 이 천 커피 수 시간 선생 자기 작품 교육 나 것...
781,순천 낙안읍성,전라여행순천 갈만한곳 낙안읍성에서 벌교까지 백마 복사 이웃추가본문 기타 기...,전라 여행 순천 갈만 한 곳 낙안 읍성 에서 벌교 까지 백마 복사 이웃 추가 본문...,읍성 낙안 순천 곳 수 꼬막 것 마을 성곽 길 풍경 여행 집 사람 이곳 봄 년 사진...
